Пет-проект: детектор спама на основе word2vec от REU Data Science Club

In [1]:
!pip install gensim

In [2]:
import kagglehub
import pandas as pd
import numpy as np
import re

## Часть 1: Загрузка данных и создание датафрейма

In [3]:
#Загружаем датасеты с Kaggle
path_1 = kagglehub.dataset_download("abdallahwagih/spam-emails") + '/spam.csv'
path_2 = kagglehub.dataset_download("uciml/sms-spam-collection-dataset") + "/spam.csv"
path_3 = kagglehub.dataset_download("ashfakyeafi/spam-email-classification") + '/email.csv'
path_4 = kagglehub.dataset_download("ozlerhakan/spam-or-not-spam-dataset") + '/spam_or_not_spam.csv'
path_5 = kagglehub.dataset_download("jackksoncsie/spam-email-dataset") + '/emails.csv'
path_6 = kagglehub.dataset_download("purusinghvi/email-spam-classification-dataset") + '/combined_data.csv'

Using Colab cache for faster access to the 'spam-emails' dataset.
Using Colab cache for faster access to the 'sms-spam-collection-dataset' dataset.
Using Colab cache for faster access to the 'spam-email-classification' dataset.
Using Colab cache for faster access to the 'spam-or-not-spam-dataset' dataset.
Using Colab cache for faster access to the 'spam-email-dataset' dataset.
Using Colab cache for faster access to the 'email-spam-classification-dataset' dataset.


In [4]:
#Формируем датафреймы
df_1 = pd.read_csv(path_1)
df_2 = pd.read_csv(path_2, encoding = 'cp1251')
df_3 = pd.read_csv(path_3)
df_4 = pd.read_csv(path_4)
df_5 = pd.read_csv(path_5)
df_6 = pd.read_csv(path_6)

####Приведем датафреймы к одному виду и объединим

In [5]:
df_1.head(3)

,Category,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...


In [6]:
df_2.head(3)

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN


In [8]:
df_2 = df_2.drop(['Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4'], axis = 1)
#Удаляем ненужные колонки
df_2.columns = ['Category', 'Message'] #Переименовываем нужные колонки
df_2.head(3)

KeyError: "['Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4'] not found in axis"

In [9]:
df_3.head(3)

,Category,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...


In [10]:
df_4.head(4)

,email,label
0,date wed NUMBER aug NUMBER NUMBER NUMBER NUMB...,0
1,martin a posted tassos papadopoulos the greek ...,0
2,man threatens explosion in moscow thursday aug...,0
3,klez the virus that won t die already the most...,0


In [11]:
df_4['label'] = df_4['label'].replace({0: 'ham', 1: 'spam'}) #Меняем разметку
df_4.columns = ['Message', 'Category'] #Меняем название колонок
df_4 = df_4.reindex(columns = ['Category', 'Message'])
df_4.head(3)

,Category,Message
0,ham,date wed NUMBER aug NUMBER NUMBER NUMBER NUMB...
1,ham,martin a posted tassos papadopoulos the greek ...
2,ham,man threatens explosion in moscow thursday aug...


In [12]:
df_5.head(3)

,text,spam
0,Subject: naturally irresistible your corporate...,1
1,Subject: the stock trading gunslinger fanny i...,1
2,Subject: unbelievable new homes made easy im ...,1


In [13]:
df_5['spam'] = df_5['spam'].replace({0: 'ham', 1: 'spam'}) #Меняем разметку
df_5.columns = ['Message', 'Category'] #Меняем название колонок
df_5 = df_5.reindex(columns = ['Category', 'Message'])
df_5.head(3)

,Category,Message
0,spam,Subject: naturally irresistible your corporate...
1,spam,Subject: the stock trading gunslinger fanny i...
2,spam,Subject: unbelievable new homes made easy im ...


In [14]:
df_6.head(3)

,label,text
0,1,ounce feather bowl hummingbird opec moment ala...
1,1,wulvob get your medircations online qnb ikud v...
2,0,computer connection from cnn com wednesday es...


In [15]:
df_6['label'] = df_6['label'].replace({0: 'ham', 1: 'spam'}) #Меняем разметку
df_6.columns = ['Category', 'Message'] #Меняем название колонок
df_6.head(3)

,Category,Message
0,spam,ounce feather bowl hummingbird opec moment ala...
1,spam,wulvob get your medircations online qnb ikud v...
2,ham,computer connection from cnn com wednesday es...


Объединим полученные датафреймы

In [16]:
df = pd.concat([df_1, df_2, df_3, df_4, df_5, df_6], axis = 0, ignore_index = True)
#Теперь у нас датафрейм с новыми индексами
df

,Category,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
108888,ham,hi given a date how do i get the last date of ...
108889,spam,now you can order software on cd or download i...
108890,spam,dear valued member canadianpharmacy provides a...
108891,ham,subscribe change profile contact us long term ...


## Часть 2: Баланс классов и предобработка текста

Оценим баланс классов, сколько объектов каждого класса у нас есть

In [17]:
df['Category'].value_counts() #Сильного дисбаланса нет

,count
Category,
ham,60873
spam,48019
"{""mode"":""full""",1


In [18]:
df.info() #Один NaN необходимо убрать

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 108893 entries, 0 to 108892
Data columns (total 2 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   Category  108893 non-null  object
 1   Message   108892 non-null  object
dtypes: object(2)
memory usage: 1.7+ MB


In [19]:
df.loc[df['Message'].isnull()] #Найдем номер строки с NaN

,Category,Message
19683,spam,NaN


In [20]:
df = df.drop(19683, axis = 0).reset_index(drop = True)
#Удаляем с обновлением индексов
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 108892 entries, 0 to 108891
Data columns (total 2 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   Category  108892 non-null  object
 1   Message   108892 non-null  object
dtypes: object(2)
memory usage: 1.7+ MB


Начнем предобработку текстов для подачи в модель

In [21]:
df_lstm = df.copy() #скопируем датафрейм для тренировки модели

def clean_text(text):
  # Удаление HTML, URL, email и цифр
  text = re.sub(r'Subject: ', '', text)
  text = re.sub(r'NUMBER', '', text)
  text = re.sub(r'<[^>]+>', '', text) #HTML
  text = re.sub(r'http\S+|www\S+', '', text) #URL
  text = re.sub(r'\S+@\S+', '', text) #email
  text = re.sub(r'[^a-zA-Z\s.,!?]', '', text)
  #Удаление спецсимволов и цифр (кроме знаков препинания)
  text = re.sub(r'([a-zA-Z])\1{2,}', r'\1', text) #Удаление повторяющихся букв
  text = text.lower() #Приведение к нижнему регистру
  text = re.sub(r'\s+', ' ', text).strip() #лишние пробелы

  return text

Посмотрим на работу функции по очистке текста на примере:

In [22]:
rand_sample = df_lstm.iloc[np.random.randint(0, len(df))]['Message']
print('Исходный текст: ', rand_sample[:100])
print('Очищенный текст: ', clean_text(rand_sample[:100]))


Исходный текст:  Subject: meeting while in houston  - - - - - - - - - - - - - - - - - - - - - - forwarded by leann wa
Очищенный текст:  meeting while in houston forwarded by leann wa


In [23]:
#Применим функцию очистки текста к нашему датафрейму
df_lstm['Message'] = df_lstm['Message'].apply(clean_text)
df_lstm.head()

,Category,Message
0,ham,"go until jurong point, crazy.. available only ..."
1,ham,ok lar... joking wif u oni...
2,spam,free entry in a wkly comp to win fa cup final ...
3,ham,u dun say so early hor... u c already then say...
4,ham,"nah i dont think he goes to usf, he lives arou..."


In [24]:
import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt_tab') #Загрузка модели токенизатора

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

Токенизация текстов в датафрейме

In [25]:
df_lstm['Message'] = df_lstm['Message'].apply(word_tokenize)
df_lstm.head()

,Category,Message
0,ham,"[go, until, jurong, point, ,, crazy, .., avail..."
1,ham,"[ok, lar, ..., joking, wif, u, oni, ...]"
2,spam,"[free, entry, in, a, wkly, comp, to, win, fa, ..."
3,ham,"[u, dun, say, so, early, hor, ..., u, c, alrea..."
4,ham,"[nah, i, dont, think, he, goes, to, usf, ,, he..."


In [26]:
df_lstm['Category'] = df_lstm['Category'].replace({'ham': 0, 'spam': 1})
#Удаляем ошибочную строку
for i in range(len(df_lstm)):
  if df_lstm['Category'][i] not in [0, 1]:
    df_lstm = df_lstm.drop(i, axis = 0)
    df = df.drop(i, axis = 0)
    # из исходного датасета тоже удаляем, он нам пригодится для ручных фичей

## Часть 3: Обучение LSTM

Подготовим эмбеддинги

In [27]:
texts = df_lstm['Message'] #массив токенизированных текстов

In [28]:
import gensim
from gensim.models import Word2Vec

#обучаем word2vec на наших текстах с нуля

model = Word2Vec(
    sentences = texts,
    vector_size = 100,
    window = 5,
    min_count = 1,
    workers = 4,
    epochs = 10
)
model.save("my_word2vec.model")

In [29]:
model = Word2Vec.load("my_word2vec.model")

# Создаем словарь с учетом <PAD> и <UNK>
word2idx = {}
word2idx["<PAD>"] = 0  # Индекс 0 для паддинга
word2idx["<UNK>"] = 1  # Индекс 1 для неизвестных слов

# Добавляем слова из модели (начиная с индекса 2)
for idx, word in enumerate(model.wv.index_to_key, start=2):
    word2idx[word] = idx

vocab_size = len(word2idx)

In [30]:
max_len = 100  # Максимальная длина последовательности

def text_to_indices(text):
    # Заменяем тексты на последовательности индексов слов из словаря
    indices = [word2idx[word] for word in text]

    # Обрезаем или дополняем паддингом
    indices = indices[:max_len] + [0] * (max_len - len(indices))
    return indices

X = np.array([text_to_indices(text) for text in texts])
y = np.array(df_lstm['Category'], dtype=np.int64)

In [31]:
embedding_dim = model.vector_size # Размер эмбеддингов
embedding_matrix = np.zeros((vocab_size, embedding_dim))
# Матрица для эмбеддингов. 1 строка - PAD, 2 - UNK.

# Заполняем матрицу эмбеддингов
for word, idx in word2idx.items():
    if idx >= 2:  # Пропускаем <PAD> и <UNK>
        if word in model.wv:
            embedding_matrix[idx] = model.wv[word]
        else:
            embedding_matrix[idx] = np.random.normal(0, 0.1, embedding_dim)

# Для UNK используем средний вектор
embedding_matrix[1] = np.mean(embedding_matrix[2:], axis=0)

In [32]:
import torch
import torch.nn as nn

class SpamLSTM(nn.Module):
    def __init__(self, embedding_matrix, hidden_dim=128):
        super().__init__()

        # Берем эмбеддинги из нашего обученного Word2Vec
        self.embedding = nn.Embedding.from_pretrained(
            torch.FloatTensor(embedding_matrix),
            padding_idx=0,
            freeze=False  # Разрешаем дообучение эмбеддингов в процессе тренировки LSTM
        )
        # LSTM-слой
        self.lstm = nn.LSTM(
            input_size=embedding_dim,
            hidden_size=hidden_dim,
            batch_first=True
        )

        self.fc = nn.Linear(hidden_dim, 1) # Преобразует выход LSTM в одно число
        self.sigmoid = nn.Sigmoid() # Функция активации для бинарной классификации

    #Зададим прямой путь через нейросеть
    def forward(self, x):
        x = self.embedding(x)
        lstm_out, _ = self.lstm(x)
        last_hidden = lstm_out[:, -1, :]
        return self.sigmoid(self.fc(last_hidden))

In [33]:
from torch.utils.data import TensorDataset, DataLoader
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y)

train_data = TensorDataset(torch.LongTensor(X_train), torch.FloatTensor(y_train))
train_loader = DataLoader(train_data, batch_size=32, shuffle=True)

In [34]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = SpamLSTM(embedding_matrix).to(device)
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

for epoch in range(10):
    for batch_x, batch_y in train_loader:
        batch_x, batch_y = batch_x.to(device), batch_y.to(device)

        optimizer.zero_grad()
        outputs = model(batch_x).squeeze(1)
        loss = criterion(outputs, batch_y)
        loss.backward()
        optimizer.step()

    print(f"Epoch {epoch}, Loss: {loss.item()}")

Epoch 0, Loss: 0.02340642921626568
Epoch 1, Loss: 0.0033707446418702602
Epoch 2, Loss: 0.055001381784677505
Epoch 3, Loss: 0.001898252754472196
Epoch 4, Loss: 0.00020942925766576082
Epoch 5, Loss: 0.00015720960800535977
Epoch 6, Loss: 0.029478464275598526
Epoch 7, Loss: 0.00023888706346042454
Epoch 8, Loss: 0.0005969381891191006
Epoch 9, Loss: 2.9503302357625216e-05


In [35]:
batch_size = 32
preds = []
with torch.no_grad():
    for i in range(0, len(X_test), batch_size):
        batch = torch.LongTensor(X_test[i:i+batch_size]).to(device)
        preds.append(model(batch).squeeze(1).cpu().numpy())
preds = np.concatenate(preds)
preds = (preds > 0.5).astype(int)
from sklearn.metrics import classification_report
print(classification_report(y_test, preds))

              precision    recall  f1-score   support

           0       0.99      0.99      0.99     15145
           1       0.98      0.98      0.98     12078

    accuracy                           0.98     27223
   macro avg       0.98      0.98      0.98     27223
weighted avg       0.98      0.98      0.98     27223



## Часть 4: Совмещение ручных фичей и предсказания LSTM

In [36]:
# Извлекаем ручные фичи из исходного датафрейма
def big_letters_counter(text):
    text_len = len(text)
    big_letters_count = sum([1 for letter in text if letter.isupper()])
    return big_letters_count / text_len
feature_1 = df['Message'].apply(big_letters_counter)
df['CAPS_Ratio'] = feature_1
df

,Category,Message,CAPS_Ratio
0,ham,"Go until jurong point, crazy.. Available only ...",0.027027
1,ham,Ok lar... Joking wif u oni...,0.068966
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,0.064516
3,ham,U dun say so early hor... U c already then say...,0.040816
4,ham,"Nah I don't think he goes to usf, he lives aro...",0.032787
...,...,...,...
108887,ham,hi given a date how do i get the last date of ...,0.000000
108888,spam,now you can order software on cd or download i...,0.000000
108889,spam,dear valued member canadianpharmacy provides a...,0.000000
108890,ham,subscribe change profile contact us long term ...,0.000000


In [37]:
feature_2 = df['Message'].apply(lambda text : text.count('!'))
df['count !'] = feature_2
df

,Category,Message,CAPS_Ratio,count !
0,ham,"Go until jurong point, crazy.. Available only ...",0.027027,0
1,ham,Ok lar... Joking wif u oni...,0.068966,0
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,0.064516,0
3,ham,U dun say so early hor... U c already then say...,0.040816,0
4,ham,"Nah I don't think he goes to usf, he lives aro...",0.032787,0
...,...,...,...,...
108887,ham,hi given a date how do i get the last date of ...,0.000000,0
108888,spam,now you can order software on cd or download i...,0.000000,0
108889,spam,dear valued member canadianpharmacy provides a...,0.000000,0
108890,ham,subscribe change profile contact us long term ...,0.000000,0


In [38]:
def indicator_words_ratio(text):
    c = 0
    for word in text.split():
        if word.lower() in ['buy', 'win', 'free', 'prize', 'click', '100%', 'discount', 'sale']:
            c += 1
    return c / len(text)

df['Indicator_words_ratio'] = df['Message'].apply(indicator_words_ratio)
df

,Category,Message,CAPS_Ratio,count !,Indicator_words_ratio
0,ham,"Go until jurong point, crazy.. Available only ...",0.027027,0,0.000000
1,ham,Ok lar... Joking wif u oni...,0.068966,0,0.000000
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,0.064516,0,0.012903
3,ham,U dun say so early hor... U c already then say...,0.040816,0,0.000000
4,ham,"Nah I don't think he goes to usf, he lives aro...",0.032787,0,0.000000
...,...,...,...,...,...
108887,ham,hi given a date how do i get the last date of ...,0.000000,0,0.000000
108888,spam,now you can order software on cd or download i...,0.000000,0,0.000000
108889,spam,dear valued member canadianpharmacy provides a...,0.000000,0,0.001873
108890,ham,subscribe change profile contact us long term ...,0.000000,0,0.000473


In [39]:
# Берем предсказание модели на этом объекте на основе индекса в матрице эмбеддингов
model.eval()  # Переводим модель в режим предсказания

batch_size = 32
preds = []
with torch.no_grad():
    for i in range(0, len(X), batch_size):
        batch = torch.LongTensor(X[i:i+batch_size]).to(device)
        preds.append(model(batch).squeeze(1).cpu().numpy())
preds = np.concatenate(preds)

In [40]:
# Совмещаем в один датафрейм
df['probs_lstm'] = preds
df = df.drop('Message', axis=1)

In [41]:
# Обучаем логистическую регрессию
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression()
X = df.drop('Category', axis=1)
y = df['Category']
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)
logreg.fit(X_train, y_train)

LogisticRegression()

In [42]:
# Считаем метрики
y_pred = logreg.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

         ham       1.00      1.00      1.00     15090
        spam       0.99      1.00      0.99     12133

    accuracy                           1.00     27223
   macro avg       1.00      1.00      1.00     27223
weighted avg       1.00      1.00      1.00     27223



# Ура, все готово 🚀